In [1]:
import pandas as pd
from pandas.plotting import lag_plot
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
import scipy
import datetime as dt
import seaborn as sns
import numpy as np
from pandas import concat
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot as plt
%matplotlib inline  

from pandas.plotting import autocorrelation_plot

# Tripoli Weather Data

- NASA/POWER SRB/FLASHFlux/MERRA2/GEOS 5.12.4 (FP-IT) 0.5 x 0.5 Degree Daily Averaged Data 
- Dates (month/day/year): 01/01/2015 through 07/19/2021 
- Location: Latitude  32.8994   Longitude 13.2026 
- Elevation from MERRA-2: Average for 1/2x1/2 degree lat/lon region = 104.28 meters   Site = na 
- Climate zone: na (reference Briggs et al: http://www.energycodes.gov) 
- Value for missing model data cannot be computed or out of model availability range: -999 


# Italy_other_Weather_Data
Parameter(s): 
- WS10M_MIN MERRA2 1/2x1/2 Minimum Wind Speed at 10 Meters (m/s) 
- WS50M_RANGE MERRA2 1/2x1/2 Wind Speed Range at 50 Meters (m/s) 
- T2M_RANGE MERRA2 1/2x1/2 Temperature Range at 2 Meters (C) 
- WS10M_RANGE MERRA2 1/2x1/2 Wind Speed Range at 10 Meters (m/s) 
- TS MERRA2 1/2x1/2 Earth Skin Temperature (C) 
- T2MWET MERRA2 1/2x1/2 Wet Bulb Temperature at 2 Meters (C) 
- T2M MERRA2 1/2x1/2 Temperature at 2 Meters (C) 
- WS10M_MAX MERRA2 1/2x1/2 Maximum Wind Speed at 10 Meters (m/s) 
- WS50M_MAX MERRA2 1/2x1/2 Maximum Wind Speed at 50 Meters (m/s) 
- T2M_MAX MERRA2 1/2x1/2 Maximum Temperature at 2 Meters (C) 
- WS50M MERRA2 1/2x1/2 Wind Speed at 50 Meters (m/s) 
- WS10M MERRA2 1/2x1/2 Wind Speed at 10 Meters (m/s) 
- T2MDEW MERRA2 1/2x1/2 Dew/Frost Point at 2 Meters (C) 
- WS50M_MIN MERRA2 1/2x1/2 Minimum Wind Speed at 50 Meters (m/s) 
- T2M_MIN MERRA2 1/2x1/2 Minimum Temperature at 2 Meters (C) 
-END HEADER-


In [2]:
weather_data = pd.read_csv("Tripoli_Weahter_Data.csv", sep = ",", dayfirst=True)

In [3]:
weather_data.head()


,LAT,LON,YEAR,MO,DY,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,WS50M_RANGE,WS10M_RANGE,WS50M_MIN,WS10M_MIN,WS50M_MAX,WS10M_MAX,WS50M,WS10M
0,32.89941,13.20261,2015,1,1,5.94,11.20,5.05,5.06,11.70,5.75,8.74,7.53,6.73,7.51,6.17,15.04,12.90,10.59,8.77
1,32.89941,13.20261,2015,1,2,2.42,13.34,7.09,7.10,13.74,11.31,12.14,3.40,2.80,7.84,5.76,11.24,8.57,9.42,7.46
2,32.89941,13.20261,2015,1,3,3.90,13.39,8.12,8.12,14.58,10.68,12.62,1.66,1.34,7.24,5.35,8.90,6.69,8.13,6.19
3,32.89941,13.20261,2015,1,4,7.52,12.98,5.82,5.84,15.92,8.40,11.94,4.28,3.92,8.19,6.49,12.47,10.41,10.89,8.92
4,32.89941,13.20261,2015,1,5,6.39,13.53,8.51,8.51,16.40,10.00,12.98,3.23,3.45,8.23,5.98,11.46,9.43,9.87,7.78


In [4]:
weather_data["Date"] = pd.to_datetime((weather_data.YEAR*10000+weather_data.MO*100+weather_data.DY).apply(str),format='%Y%m%d')
weather_data = weather_data.set_index("Date")
weather_data = weather_data.drop(["LON","LAT","MO","YEAR","DY"], axis = 1)
weather_data.index = pd.to_datetime(weather_data.index)

In [5]:
weather_data

,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,WS50M_RANGE,WS10M_RANGE,WS50M_MIN,WS10M_MIN,WS50M_MAX,WS10M_MAX,WS50M,WS10M
Date,,,,,,,,,,,,,,,
2015-01-01,5.94,11.20,5.05,5.06,11.70,5.75,8.74,7.53,6.73,7.51,6.17,15.04,12.90,10.59,8.77
2015-01-02,2.42,13.34,7.09,7.10,13.74,11.31,12.14,3.40,2.80,7.84,5.76,11.24,8.57,9.42,7.46
2015-01-03,3.90,13.39,8.12,8.12,14.58,10.68,12.62,1.66,1.34,7.24,5.35,8.90,6.69,8.13,6.19
2015-01-04,7.52,12.98,5.82,5.84,15.92,8.40,11.94,4.28,3.92,8.19,6.49,12.47,10.41,10.89,8.92
2015-01-05,6.39,13.53,8.51,8.51,16.40,10.00,12.98,3.23,3.45,8.23,5.98,11.46,9.43,9.87,7.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-15,6.71,29.46,19.01,23.26,31.35,24.64,27.51,3.97,4.08,2.09,1.48,6.06,5.56,4.88,4.25
2021-07-16,9.12,29.74,18.33,23.02,33.09,23.97,27.71,6.16,5.89,0.80,0.68,6.96,6.57,3.77,3.38
2021-07-17,5.87,28.34,18.05,22.16,29.68,23.81,26.28,7.14,6.76,1.21,0.95,8.35,7.71,5.18,4.49


In [6]:
daily_arrivals = pd.read_csv("Sea arrivals by day_ITA.csv", sep = ";", index_col=0,parse_dates=True, header=0,dayfirst=True)

In [7]:
df_merged = daily_arrivals.merge(weather_data, how='outer', left_index=True, right_index=True)


In [8]:
df_merged = df_merged.dropna()

In [9]:
df_merged

,individuals,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,WS50M_RANGE,WS10M_RANGE,WS50M_MIN,WS10M_MIN,WS50M_MAX,WS10M_MAX,WS50M,WS10M
Date,,,,,,,,,,,,,,,,
2015-10-01,343.0,7.63,24.87,18.40,18.40,28.77,21.14,24.43,7.24,6.07,1.31,1.04,8.54,7.11,5.27,4.18
2015-10-03,128.0,7.25,25.00,20.26,20.26,28.22,20.97,24.05,4.34,2.52,3.15,2.99,7.49,5.51,4.78,3.73
2015-10-04,62.0,5.20,25.12,21.38,21.38,27.46,22.26,24.42,3.67,3.26,1.00,0.89,4.66,4.14,3.19,2.60
2015-10-06,2001.0,7.76,25.33,17.81,17.81,29.75,21.99,25.79,5.62,3.52,1.08,0.85,6.70,4.37,4.27,3.17
2015-10-07,1010.0,7.37,25.88,19.14,19.14,30.73,23.35,26.16,3.38,2.06,4.83,3.55,8.21,5.61,6.35,4.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-01,322.0,11.61,33.06,14.31,23.68,39.20,27.59,33.06,9.90,6.17,0.67,0.55,10.57,6.71,6.99,5.03
2021-07-02,12.0,10.91,31.92,18.07,24.33,36.92,26.01,30.59,5.25,4.81,0.34,0.31,5.59,5.11,3.25,2.73
2021-07-03,308.0,10.46,31.98,16.90,23.85,36.84,26.38,30.80,4.50,3.38,1.68,1.26,6.18,4.63,3.98,3.05


In [ ]:
corr = df_merged.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()